# Reinforcement Learning Final Project: Exploration Strategies for Robotic Manipulation

### Import helper libraries

In [6]:
### Important trick to keep JupyterLab from crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import gym
import sys
import traceback
from arguments import get_args
import random
import torch
import tqdm
from datetime import datetime
from mpi4py import MPI
from mpi_utils.mpi_utils import sync_networks, sync_grads
from rl_modules.replay_buffer import replay_buffer
from rl_modules.models import actor, critic, noisy_actor, noisy_critic
from mpi_utils.normalizer import normalizer
from her_modules.her import her_sampler
import copy
import math
from typing import Dict, List, Tuple, Callable
from collections import namedtuple
from copy import deepcopy
import ipywidgets as widgets
import matplotlib.pyplot as plt
import more_itertools as mitt
import pygame
import glfw
from math import floor
from tiling import IHT
from pathlib import Path
from schedule import ExponentialSchedule, OUSchedule

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [36, 4]
fullPath = str(Path('.').absolute())

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['IN_MPI'] = '1'

### Create environments and place to store model weights
envs = {
    'push': {
        'model': gym.make('FetchPush-v1'),
        'weights': None
    },
    'reach': {
        'model': gym.make('FetchReach-v1'),
        'weights': None
    },
    'slide': {
        'model': gym.make('FetchSlide-v1'),
        'weights': None
    },
    'pick': {
        'model': gym.make('FetchPickAndPlace-v1'),
        'weights': None
    }
}

### Graphically Render Policy

Using Mujoco backend

In [7]:
def render(env, policy=None):
    """Graphically render an episode using the given policy

    :param env:  Gym environment
    :param policy:  function which maps state to action.  If None, the random
                    policy is used.
    """
    glfw.init()
    if policy is None:

        def policy(state):
            return env.action_space.sample()

    state = env.reset()
    env.render()
    i = 0
    while i < 3000:
        action = policy(state)
        state, _, done, _ = env.step(action)
        env.render()
        if done:
            break
        i += 1
            
    env.close()
    glfw.terminate()

### Helper Functions

In [8]:
def preproc_og(o, g):
    o = np.clip(o, -200, 200)
    g = np.clip(g, -200, 200)
    return o, g

# pre_process the inputs
def preproc_inputs(obs, g):
    obs_norm = self.o_norm.normalize(obs)
    g_norm = self.g_norm.normalize(g)
    # concatenate the stuffs
    inputs = np.concatenate([obs_norm, g_norm])
    inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(0)
    return inputs

### Define Structure for Deep Deterministic Policy Gradient (DDPG) Agent

In [56]:
"""
ddpg with HER (MPI-version)

"""
class ddpg_agent:
    def __init__(self, args, env, env_params):
        self.args = args
        self.env = env
        self.env_params = env_params
        # create the network
        self.actor_network = noisy_actor(env_params) if bool(self.args.noisy) else actor(env_params)
        self.critic_network = noisy_critic(env_params) if bool(self.args.noisy) else critic(env_params) 
        # sync the networks across the cpus
        sync_networks(self.actor_network)
        sync_networks(self.critic_network)
        # build up the target network
        self.actor_target_network = actor(env_params)
        self.critic_target_network = critic(env_params)
        # load the weights into the target networks
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        self.critic_target_network.load_state_dict(self.critic_network.state_dict())
        # if use gpu
        if self.args.cuda:
            self.actor_network.cuda()
            self.critic_network.cuda()
            self.actor_target_network.cuda()
            self.critic_target_network.cuda()
        # create the optimizer
        self.actor_optim = torch.optim.Adam(self.actor_network.parameters(), lr=self.args.lr_actor)
        self.critic_optim = torch.optim.Adam(self.critic_network.parameters(), lr=self.args.lr_critic)
        # her sampler
        self.her_module = her_sampler(self.args.replay_strategy, self.args.replay_k, self.env.compute_reward)
        # create the replay buffer
        self.buffer = replay_buffer(self.env_params, self.args.buffer_size, self.her_module.sample_her_transitions)
        # create the normalizer
        self.o_norm = normalizer(size=env_params['obs'], default_clip_range=self.args.clip_range)
        self.g_norm = normalizer(size=env_params['goal'], default_clip_range=self.args.clip_range)
        ########### Setup exploration parameters ########
        # Create exploration parameters
        self.epsilon = self.args.epsilon
        # Create e-greedy schedule
        self.e_schedule = ExponentialSchedule(1.0,0.01,self.args.n_cycles)
        self.ou_schedule = OUSchedule(self.args.n_cycles,4)
        # Create store for count based methods
        self.num_tiles = 10
        self.max_size = 1000
        self.iht = IHT(self.max_size)
        self.weights_fq = np.zeros(self.max_size)
        self.weights_ucb = np.zeros(self.max_size)
        # Create strategy store
        self.strategies = {
        'no_explore': self.no_explore,
        'standard': self.standard_explore,
        'e_greedy': self.e_greedy,
        'e_greedy_decay': self.e_greedy_decay,
        'ucb': self.ucb,
        'ou': self.ornstein_uhlenbeck,
        'count': self.frequency_explore
        }
        # Select the exploration strategy
        self.explore_strategy = self.strategies[self.args.strategy]

    def learn(self):
        """
        train the network

        """
        # start to collect samples
        pbar = tqdm.notebook.trange(self.args.n_epochs)
        for epoch in pbar:
            # Reset process noise
            self.ou_schedule.reset()
            # Reset count based method stores
            self.iht = IHT(self.max_size)
            self.weights_fq = np.zeros(self.max_size)
            self.weights_ucb = np.zeros(self.max_size)
            for cycle_num in range(self.args.n_cycles):
                mb_obs, mb_ag, mb_g, mb_actions = [], [], [], []
                for _ in range(self.args.num_rollouts_per_mpi):
                    # reset the rollouts
                    ep_obs, ep_ag, ep_g, ep_actions = [], [], [], []
                    # reset the environment
                    observation = self.env.reset()
                    obs = observation['observation']
                    ag = observation['achieved_goal']
                    g = observation['desired_goal']
                    # start to collect samples
                    for t in range(self.env_params['max_timesteps']):
                        with torch.no_grad():
                            input_tensor = self._preproc_inputs(obs, g)
                            pi = self.actor_network(input_tensor)
                            action = self._select_actions(pi, input_tensor, cycle_num)
                        # feed the actions into the environment
                        observation_new, _, _, info = self.env.step(action)
                        obs_new = observation_new['observation']
                        ag_new = observation_new['achieved_goal']
                        # append rollouts
                        ep_obs.append(obs.copy())
                        ep_ag.append(ag.copy())
                        ep_g.append(g.copy())
                        ep_actions.append(action.copy())
                        # re-assign the observation
                        obs = obs_new
                        ag = ag_new
                    ep_obs.append(obs.copy())
                    ep_ag.append(ag.copy())
                    mb_obs.append(ep_obs)
                    mb_ag.append(ep_ag)
                    mb_g.append(ep_g)
                    mb_actions.append(ep_actions)
                # convert them into arrays
                mb_obs = np.array(mb_obs)
                mb_ag = np.array(mb_ag)
                mb_g = np.array(mb_g)
                mb_actions = np.array(mb_actions)
                # store the episodes
                self.buffer.store_episode([mb_obs, mb_ag, mb_g, mb_actions])
                self._update_normalizer([mb_obs, mb_ag, mb_g, mb_actions])
                for _ in range(self.args.n_batches):
                    # train the network
                    self._update_network()
                # soft update
                self._soft_update_target_network(self.actor_target_network, self.actor_network)
                self._soft_update_target_network(self.critic_target_network, self.critic_network)
            # start to do the evaluation
            success_rate = self._eval_agent()
            if MPI.COMM_WORLD.Get_rank() == 0:
                print('[{}] epoch is: {}, eval success rate is: {:.3f}'.format(datetime.now(), epoch, success_rate))

        # Return the network weights
        #torch.save([self.o_norm.mean, self.o_norm.std, self.g_norm.mean, self.g_norm.std, self.actor_network.state_dict()], \
        #    self.model_path + '/model.pt')
        return self.actor_network, self.o_norm, self.g_norm
    

    def get_active_tiles(self, state, action):
        active_tiles = self.tiles(self.iht, self.num_tiles, np.concatenate((state,action)))
        return active_tiles

    def hash_coords(self, coordinates, m, read_only=False):
        if isinstance(m, IHT): return m.get_index(tuple(coordinates), read_only)
        if isinstance(m, int): return hash(tuple(coordinates)) % m
        if m is None: return coordinates

    def tiles(self, iht_or_size, num_tilings, floats, ints=None, read_only=False):
        """returns num-tilings tile indices corresponding to the floats and ints"""
        if ints is None:
            ints = []
        qfloats = [floor(f * num_tilings) for f in floats]
        tiles = []
        for tiling in range(num_tilings):
            tilingX2 = tiling * 2
            coords = [tiling]
            b = tiling
            for q in qfloats:
                coords.append((q + b) // num_tilings)
                b += tilingX2
            coords.extend(ints)
            tiles.append(self.hash_coords(coords, iht_or_size, read_only))
        return tiles

    # no exploration, returns the action
    def no_explore(self, action, state, step):
        return action
        
    # run basic e-greedy exploration
    def e_greedy(self, action, state, step):
        if np.random.random() < self.epsilon:
            # Select a random action state
            action = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        return action
        
    # Use decaying e-greedy strategy
    def e_greedy_decay(self, action, state, step):
        if np.random.random() < self.e_schedule.value(step):
            # Select a random action state
            action = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        return action
    
    # Use upper confidence bound
    def ucb(self, action, state, step):
        active = self.get_active_tiles(np.random.normal(state.numpy().flatten()),np.random.normal(action))
        self.weights_ucb[active] += 1
        additive = np.sqrt(np.log(np.sum(self.weights_ucb[active]))/np.sum(self.weights_ucb))
        return action + np.random.normal(loc=0.0,scale=additive,size=4)
    
    # Select action as a function of how frequently they have occured
    def frequency_explore(self, action, state, step):
        beta = 0.5
        active = self.get_active_tiles(np.random.normal(state.numpy().flatten()),np.random.normal(action))
        self.weights_fq[active] += 1
        additive = beta/np.sqrt(np.sum(self.weights_fq[active]))
        return action + np.random.normal(loc=0.0,scale=additive,size=4)

    # White noise based exploration
    def standard_explore(self, action, state, step):
        # random actions...
        random_actions = np.random.uniform(low=-self.env_params['action_max'], high=self.env_params['action_max'], \
                                            size=self.env_params['action'])
        # choose if use the random actions
        return action + np.random.binomial(1, self.args.random_eps, 1)[0] * (random_actions - action)
    
    # Explore using OU process
    def ornstein_uhlenbeck(self, action, state, step):
        return action + self.ou_schedule.value(step)
    
    # pre_process the inputs
    def _preproc_inputs(self, obs, g):
        obs_norm = self.o_norm.normalize(obs)
        g_norm = self.g_norm.normalize(g)
        # concatenate the stuffs
        inputs = np.concatenate([obs_norm, g_norm])
        inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(0)
        if self.args.cuda:
            inputs = inputs.cuda()
        return inputs
    
    # this function will choose action for the agent and do the exploration
    def _select_actions(self, pi, state, step):
        action = pi.cpu().numpy().squeeze()
        # add the gaussian
        #action += self.args.noise_eps * self.env_params['action_max'] * np.random.randn(*action.shape)
        #action = np.clip(action, -self.env_params['action_max'], self.env_params['action_max'])
        action = self.explore_strategy(action,state,step)
        action = np.clip(action, -self.env_params['action_max'], self.env_params['action_max'])
        return action

    # update the normalizer
    def _update_normalizer(self, episode_batch):
        mb_obs, mb_ag, mb_g, mb_actions = episode_batch
        mb_obs_next = mb_obs[:, 1:, :]
        mb_ag_next = mb_ag[:, 1:, :]
        # get the number of normalization transitions
        num_transitions = mb_actions.shape[1]
        # create the new buffer to store them
        buffer_temp = {'obs': mb_obs, 
                       'ag': mb_ag,
                       'g': mb_g, 
                       'actions': mb_actions, 
                       'obs_next': mb_obs_next,
                       'ag_next': mb_ag_next,
                       }
        transitions = self.her_module.sample_her_transitions(buffer_temp, num_transitions)
        obs, g = transitions['obs'], transitions['g']
        # pre process the obs and g
        transitions['obs'], transitions['g'] = preproc_og(obs, g)
        # update
        self.o_norm.update(transitions['obs'])
        self.g_norm.update(transitions['g'])
        # recompute the stats
        self.o_norm.recompute_stats()
        self.g_norm.recompute_stats()

    # soft update
    def _soft_update_target_network(self, target, source):
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_((1 - self.args.polyak) * param.data + self.args.polyak * target_param.data)

    # update the network
    def _update_network(self):
        # sample the episodes
        transitions = self.buffer.sample(self.args.batch_size)
        # pre-process the observation and goal
        o, o_next, g = transitions['obs'], transitions['obs_next'], transitions['g']
        transitions['obs'], transitions['g'] = preproc_og(o, g)
        transitions['obs_next'], transitions['g_next'] = preproc_og(o_next, g)
        # start to do the update
        obs_norm = self.o_norm.normalize(transitions['obs'])
        g_norm = self.g_norm.normalize(transitions['g'])
        inputs_norm = np.concatenate([obs_norm, g_norm], axis=1)
        obs_next_norm = self.o_norm.normalize(transitions['obs_next'])
        g_next_norm = self.g_norm.normalize(transitions['g_next'])
        inputs_next_norm = np.concatenate([obs_next_norm, g_next_norm], axis=1)
        # transfer them into the tensor
        inputs_norm_tensor = torch.tensor(inputs_norm, dtype=torch.float32)
        inputs_next_norm_tensor = torch.tensor(inputs_next_norm, dtype=torch.float32)
        actions_tensor = torch.tensor(transitions['actions'], dtype=torch.float32)
        r_tensor = torch.tensor(transitions['r'], dtype=torch.float32) 
        if self.args.cuda:
            inputs_norm_tensor = inputs_norm_tensor.cuda()
            inputs_next_norm_tensor = inputs_next_norm_tensor.cuda()
            actions_tensor = actions_tensor.cuda()
            r_tensor = r_tensor.cuda()
        # calculate the target Q value function
        with torch.no_grad():
            # do the normalization
            # concatenate the stuffs
            actions_next = self.actor_target_network(inputs_next_norm_tensor)
            q_next_value = self.critic_target_network(inputs_next_norm_tensor, actions_next)
            q_next_value = q_next_value.detach()
            target_q_value = r_tensor + self.args.gamma * q_next_value
            target_q_value = target_q_value.detach()
            # clip the q value
            clip_return = 1 / (1 - self.args.gamma)
            target_q_value = torch.clamp(target_q_value, -clip_return, 0)
        # the q loss
        real_q_value = self.critic_network(inputs_norm_tensor, actions_tensor)
        critic_loss = (target_q_value - real_q_value).pow(2).mean()
        # the actor loss
        actions_real = self.actor_network(inputs_norm_tensor)
        actor_loss = -self.critic_network(inputs_norm_tensor, actions_real).mean()
        actor_loss += self.args.action_l2 * (actions_real / self.env_params['action_max']).pow(2).mean()
        # start to update the network
        self.actor_optim.zero_grad()
        actor_loss.backward()
        sync_grads(self.actor_network)
        self.actor_optim.step()
        # update the critic_network
        self.critic_optim.zero_grad()
        critic_loss.backward()
        sync_grads(self.critic_network)
        self.critic_optim.step()

    # do the evaluation
    def _eval_agent(self):
        total_success_rate = []
        for _ in range(self.args.n_test_rollouts):
            per_success_rate = []
            observation = self.env.reset()
            obs = observation['observation']
            g = observation['desired_goal']
            for _ in range(self.env_params['max_timesteps']):
                with torch.no_grad():
                    input_tensor = self._preproc_inputs(obs, g)
                    pi = self.actor_network(input_tensor)
                    # convert the actions
                    actions = pi.detach().cpu().numpy().squeeze()
                observation_new, _, _, info = self.env.step(actions)
                obs = observation_new['observation']
                g = observation_new['desired_goal']
                per_success_rate.append(info['is_success'])
            total_success_rate.append(per_success_rate)
        total_success_rate = np.array(total_success_rate)
        local_success_rate = np.mean(total_success_rate[:, -1])
        global_success_rate = MPI.COMM_WORLD.allreduce(local_success_rate, op=MPI.SUM)
        return global_success_rate / MPI.COMM_WORLD.Get_size()


### Define environmental parameters

In [57]:
def get_env_params(env):
    obs = env.reset()
    # close the environment
    params = {'obs': obs['observation'].shape[0],
            'goal': obs['desired_goal'].shape[0],
            'action': env.action_space.shape[0],
            'action_max': env.action_space.high[0],
            }
    params['max_timesteps'] = env._max_episode_steps
    return params


### Parse arguments and run code

In [58]:
def launch(args, path=None):
    # Save path
    save_path = os.path.join(path,f'{args.env_name}.wts')
    # create the ddpg_agent
    env = gym.make(args.env_name)
    # get the environment parameters
    env_params = get_env_params(env)
    # create the ddpg agent to interact with the environment 
    ddpg_trainer = ddpg_agent(args, env, env_params)
    agent_weights, onorm, gnorm = ddpg_trainer.learn()
    # Save weights here
    # torch.save(agent_weights.state_dict(), save_path)
    # Save extra essential pieces
    torch.save([onorm.mean, onorm.std, gnorm.mean, gnorm.std, agent_weights.state_dict()], save_path)
    return agent_weights, onorm, gnorm

### TODO: Cleanup the way code is launched

In [60]:
args_to_parse = '--env-name FetchReach-v1 --n-epochs=50 --strategy=ucb --noisy=1'
args = get_args(args_to_parse)
out = launch(args, fullPath)

  0%|          | 0/50 [00:00<?, ?it/s]

IHT full, starting to allow collisions
[2021-12-06 18:10:46.768710] epoch is: 0, eval success rate is: 0.500
IHT full, starting to allow collisions
[2021-12-06 18:11:04.378719] epoch is: 1, eval success rate is: 1.000
IHT full, starting to allow collisions


KeyboardInterrupt: 

In [ ]:
#  Jupyter UI - Trigger policies for individual agents
def button_callback(button):
    for b in buttons:
        b.disabled = True

    env = envs[button.description]['model']
    final_policy = None
    try:
        env_name = envs[button.description]['model'].env.spec.id
        env_params = get_env_params(env)
        model = torch.load(f'{env_name}.wts')
        model_weights = model[4]
        actor_model = actor(env_params)
        actor_model.load_state_dict(model_weights)
        # Create policy using function
        def policy(state):
            # reset the environment
            obs = state['observation']
            g = state['desired_goal']
            o_norm = normalizer(size=10, default_clip_range=200)
            o_norm.mean = model[0]
            o_norm.std = model[1]
            g_norm = normalizer(size=3, default_clip_range=200)
            g_norm.mean = model[2]
            g_norm.std = model[3]
            obs_norm = o_norm.normalize(obs)
            g_norm = g_norm.normalize(g)
            # concatenate the stuffs
            input_tensor = np.concatenate([obs_norm, g_norm])
            input_tensor = torch.tensor(input_tensor, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                pi = actor_model(input_tensor)
            action = pi.cpu().numpy().squeeze()
            # add the gaussian
            action = np.clip(action, -1, 1)
            return action
        final_policy = policy
    except Exception as e:
        print('Could not create policy - running random policy')
        print(str(e))
        traceback.print_exc()
        return
    render(env, final_policy)
    env.close()
        
    for b in buttons:
        b.disabled = False

buttons = []
for env_id in envs.keys():
    button = widgets.Button(description=env_id)
    button.on_click(button_callback)
    buttons.append(button)

print('Click a button to evaluate a policy')
b = widgets.HBox(buttons)
display(b)

In [ ]:
num_tiles = 10
max_size = 1000
iht = IHT(max_size)

def get_active_tiles(state, action):
    active_tiles = tiles(iht, num_tiles, np.concatenate((state,action)))
    return active_tiles

def hash_coords(coordinates, m, read_only=False):
    if isinstance(m, IHT): return m.get_index(tuple(coordinates), read_only)
    if isinstance(m, int): return hash(tuple(coordinates)) % m
    if m is None: return coordinates

def tiles(iht_or_size, num_tilings, floats, ints=None, read_only=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    if ints is None:
        ints = []
    qfloats = [floor(f * num_tilings) for f in floats]
    tiles = []
    for tiling in range(num_tilings):
        tilingX2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // num_tilings)
            b += tilingX2
        coords.extend(ints)
        tiles.append(hash_coords(coords, iht_or_size, read_only))
    return tiles

In [ ]:
s = envs['push']['model'].reset()
weights = np.zeros(max_size)

In [ ]:
t = []
ucb = []
beta = 0.5
for x in range(10000):
    active = get_active_tiles(np.random.normal(size=25),np.random.normal(size=3))
    weights[active] += 1
    additive = beta/np.sqrt(np.sum(weights[active]))
    t.append(additive)
    ucb.append(np.sqrt(np.log(np.sum(weights[active]))/np.sum(weights)))

In [ ]:
plt.plot(t,label='count')
plt.plot(ucb,label='ucb')
plt.legend()